## Аггрегация разметки датасета ruModAr
*Subset edition*

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

В разметке было 1800 заданий: по 600 заданий на каждую математическую операцию и по 300 на две опции:
- можно символ `->` заменить на просто `=`
- можно символ `->` заменить на ` + 1 =`

Разметчики делали выбор, и записывали свое решение. Проверяется не столько способность решить пример, сколько умение логически размышлять и делать вывод из контекста - уже решенных примеров.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

### Сбор данных разметки и фильтрация разметчиков

Данный датасет представляет из себя лишь подмножество, которое было специально сгенерировано. Это не тестовый сет датасета из мультимодального бенчмарка (инструкционный сет)! Размер выборки - 1800 объектов.

In [2]:
assignments = pd.read_csv('assignments_from_pool_41461361__28-09-2023.tsv', sep='\t')

Разметчикам предлагалось на основании контекста из решенных пяти примеров и одного нерешенного примера ответить на вопрос, чему равен нерешенный пример, если заменить в нем специальный символ `->` соответственно контексту.
Вход: 
- INPUT:context (пример: `2 + 2 -> 5`).
- INPUT:problem (пример: `3 + 3 ->`).

Выход:
- OUTPUT:solution (целое число, например: `7`).

In [10]:
assignments.head(1)

,INPUT:context,INPUT:problem,OUTPUT:solution,GOLDEN:solution,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,48 * 22 -> 1057\r\n7 * 23 -> 162\r\n56 * 68 ->...,80 * 35 ->,2801,NaN,NaN,NaN,https://platform.toloka.ai/task/41461361/00027...,000278a671--6514b53aaa6bb16340de1f8c,000278a671--651555040b89303759a5c4f3,000278a671--651555040b89303759a5c4f0,1d09bb1b80a54c03c195eb8af088f05b,APPROVED,2023-09-28T10:27:16.309,2023-09-28T10:31:26.669,2023-09-28T10:31:26.669,0.022


Приводим ответы к универсальному виду.

In [5]:
assignments['OUTPUT:solution'] = assignments['OUTPUT:solution'].astype(int)

Пока в ответах есть Nan'ы, нельзя сделать ответы интами, потому будут строками.

In [6]:
assignments['GOLDEN:solution'] = assignments['GOLDEN:solution'].apply(lambda x: x if str(x) == 'nan' else str(int(x)))

Фильтруем толокеров с `accuracy < 0.5` на контрольных заданиях, чтобы не учитывать их ответы при подсчете метрик.

In [7]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    context = row[0]
    problem = row[1]

    out = row[2]
    
    gold = row[3]

    user = row[10]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  251
Bad users: 5


5 из 251 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Отделяем контроль от основы, так как контрольные задания создавались отдельно и не должны учитываться при подсчете метрик. На контрольных заданиях есть `GOLDEN:solution`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:context` содержит NaN.

In [8]:
assignments_no_control = assignments[assignments['GOLDEN:solution'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:context'].notnull()]

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [11]:
from collections import defaultdict

text_dict = defaultdict(list)

for context, problem, user, out in zip(
    assignments_no_control_no_null["INPUT:context"], assignments_no_control_no_null["INPUT:problem"], 
    assignments_no_control_no_null["ASSIGNMENT:worker_id"], assignments_no_control_no_null["OUTPUT:solution"]
    ):
    if user not in bad_users:
        text_dict[(context, problem)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

1800


In [12]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 1774, 4: 26})

Есть 26 заданий, где перекрытие 4. Значит, для этих заданий порогом будет большинство в 3 голоса (порог аналогичен перекрытию 5).

In [13]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    if len(lst) == 5:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most >= 3:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res
    elif len(lst) == 4:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most > 2:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res

In [14]:
len(preds_full)

1794

Отсеялись 6 заданий, где большинства не набралось.

In [15]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['context', 'problem']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными ответами.

In [16]:
res_df = pd.read_csv('exp_wa.tsv', sep='\t')

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [17]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['INPUT:context'] = res_df['INPUT:context'].apply(format_text)
res_df['INPUT:problem'] = res_df['INPUT:problem'].apply(format_text)

preds_full_df['context'] = preds_full_df['context'].apply(format_text)
preds_full_df['problem'] = preds_full_df['problem'].apply(format_text)

res_df = res_df.rename({
    'INPUT:context': 'context',
    'INPUT:problem': 'problem',
    'outputs': 'lb'
}, axis=1)

res_df['full'] = res_df['context'] + ' ' + res_df['problem']
preds_full_df['full'] = preds_full_df['context'] + ' ' + preds_full_df['problem']

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [18]:
new = res_df.merge(preds_full_df.drop(['context', 'problem'], axis=1), on='full', how='left')

In [19]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

1794

Ни одна строка не была утеряна.

In [20]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [47]:
new_valid.head(1)

,lb_x,context,problem,full,lb_y
0,874,199 + 998 -> 1197 191 + 519 -> 710 557 + 78 ->...,31 + 843 ->,199 + 998 -> 1197 191 + 519 -> 710 557 + 78 ->...,874


### Подсчет метрики

Если в правом столбце меток осталось 1794 непустых строк, значит, форматирование было подчищено корректно и ничего не потерялось

Попробуем посчитать разные метрики

In [152]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.9994425863991081

`Accuracy = 0.999`

### Анализ ошибок

Посмотрим на ошибки в голосовании большинством. Сначала добавим указание на части датасета.

In [22]:
(new_valid['lb_x'] != new_valid['lb_y']).sum()

1

Разметчики почти идеально решили задачу. Только в одном задании большинство ошиблось.

In [23]:
new_valid[new_valid['lb_x'] != new_valid['lb_y']]

,lb_x,context,problem,full,lb_y
1164,-445,366 - 677 -> -310 797 - 428 -> 370 322 - 621 -...,25 - 471 ->,366 - 677 -> -310 797 - 428 -> 370 322 - 621 -...,-447


Очевидно, разметчики верно поняли, что тут специальный символ меняет результат на единицу, но перепутали и вместо прибавления единицы вычли ее. Больше анализировать в обработанных данных нечего - ошибок больше не осталось. Нужно смотреть на полный пул отдельных ответов разметчиковна каждое задание.

In [24]:
new = assignments_no_control_no_null.iloc[:, [0, 1, 2, 10]].rename({
    'INPUT:context': 'context',
    'INPUT:problem': 'problem',
    'OUTPUT:solution': 'label',
    'ASSIGNMENT:worker_id': 'user'
}, axis=1).copy()

Соединяем контекст и нерешенный пример, чтобы было удобно соединить с правильными ответами.

In [25]:
new['full'] = new.context + ' ' + new.problem
new['full'] = new['full'].apply(format_text)

In [29]:
new = new.merge(res_df.drop(['context', 'problem'], axis=1), on='full', how='left')

Соединяем с правильными ответами по полному тексту задания.

In [30]:
new['sign'] = new.problem.apply(lambda x: x.split()[1])

Посмотрим на разницу и абсолютную разницу ответов разметчиков и правильных ответов.

In [31]:
new['diff'] = new['label'] - new['lb']
new['abs_diff'] = new['diff'].abs()

Ошибками считаем те строки, где разность не равна нулю.

In [32]:
new['error'] = (new['diff'] != 0).astype(int)

In [34]:
new.head(1)

,context,problem,label,user,full,sign,lb,diff,abs_diff,error
0,48 * 22 -> 1057\r\n7 * 23 -> 162\r\n56 * 68 ->...,80 * 35 ->,2801,1d09bb1b80a54c03c195eb8af088f05b,48 * 22 -> 1057 7 * 23 -> 162 56 * 68 -> 3809 ...,*,2801,0,0,0


Отделим ошибки.

In [35]:
errors = new[new['error'] == 1].copy()

In [36]:
errors['diff'].value_counts()

diff
-2      116
-1       76
 1       25
 100      7
-3        5
       ... 
 504      1
 96       1
-30       1
 78       1
 640      1
Name: count, Length: 168, dtype: int64

Наиболее часто встречающаяся ошибка у толокеров, как и следовало ожидать, это просчет на -2. Природа этого просчета в простой арифметике. По заданию специальный символ может превращаться в `+ 1 =`, значит необходимо к результату вычисления прибавить единицу. В случае отрицательных чисел прибавление единицы число по модулю уменьшает. Отсюда и ошибка. Пример:

`-10 + 1 = -9`

`10 + 1 = 11`

А толокеры могут путать правила и делать так: 

`-10 + 1 = -11`

In [37]:
errors[errors['abs_diff'] == 2]['sign'].value_counts()

sign
-    89
+    17
*    11
Name: count, dtype: int64

Набиольшее количество таких ошибок допущенно в примерах с вычитанием, так как именно там много отрицательных чисел.

In [38]:
errors[(errors['abs_diff'] == 2) & (errors['lb'] < 0)]['sign'].value_counts()

sign
-    57
Name: count, dtype: int64

Что и требовалось доказать! Наиболее частая ошибка ярче всего проявляется в примерах с вычитанием. В большинстве случаев такая ошибка делается одним или двумя разметчиками, что не приводит к ее появлению при голосовании большинством. Однако в одном примере собралось сразу 3 разметчика, которые сделали одинаковую ошибку.

In [39]:
errors['sign'].value_counts()

sign
-    225
+    120
*     83
Name: count, dtype: int64

В целом, среди всех ошибок чаще всего разметчики ошибаются именно в примерах с вычитанием.

Теперь оценим вероятность ошибки грубо по всему пулу без заведомо "плохих" разметчиков.

In [46]:
pd.DataFrame({
    "Сложение": new[(new['error'] == 1) & (new['sign'] == '+')].shape[0] / len(new),
    "Вычитание": new[(new['error'] == 1) & (new['sign'] == '-')].shape[0] / len(new),
    "Умножение": new[(new['error'] == 1) & (new['sign'] == '*')].shape[0] / len(new),
}, index=['Доля ошибок от всего пула'])

,Сложение,Вычитание,Умножение
Доля ошибок от всего пула,0.013333,0.025,0.009222


Очевидно, вероятность ошибки на примере с вычитанием в 2 раза выше, чем на примере со сложением и почти в 3 раза больше, чем в примерах с умножением.